## References

1.	Abstractive Text Summarization: https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/
2.	Extractive Text Summarization: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import copy
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import tensorflow as tf
import tensorflow_hub as hub
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sandeep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sandeep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# loading the universal sentence encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)

In [8]:
# df = pd.read_csv('/content/drive/MyDrive/Text Summarization/tennis_articles.csv', encoding='latin')
# df.shape

In [ ]:
df.head(1)

## Data Preparation:
        - Tokenize the entire document/documents by sentence.
        - Flatten the tokenized sentences ie [['i am good'], ['i am going out']] becomes ['i amd good', 'i am going out']
        - If necessary, do data cleaning such as removal of punctuations, change to lower case, remove numerics, stopwords, etc.

In [9]:
def data_preparation(dfr, text_col):
    
    # sentence tokenizer
    sent_tokens = []
    for s in dfr[text_col]:
        sents = nltk.sent_tokenize(s)
        sent_tokens.append(sents)

    # flatten the list of list to get single list
    sent_tokens = [y for x in sent_tokens for y in x]

    # create a dictionary with sentences as key and integers as values
    # this dictionary will be used in later section to get the top n sentences as per document order
    sent_as_key = dict(zip(sent_tokens, np.arange(1, len(sent_tokens)+1)))

    return sent_tokens, sent_as_key

## Vectorize the sentences using universal sentence encoder
        - Vectorize each sentences and store in a list

In [10]:
def embed(sent_tokens):

    # list to store all sentence vectors
    sent_vectors = []
    for s in sent_tokens:
        sent_vectors.append(model([s]))

    return sent_vectors

## Get sentence similarity between sentences with cosine similarity
        - Create a matrix of NxN where N is the number of sentences
        - Get the cosine similarity between each sentences.
        - Similarity between same sentence will be 0 only.

In [11]:
def cosine_similarity_matrix(sent_tokens, sent_vectors):

    cosine_matrix = np.zeros([len(sent_tokens), len(sent_tokens)])

    for i in range(len(sent_tokens)):
        for j in range(len(sent_tokens)):
            if i != j:
                cosine_matrix[i][j] = cosine_similarity(sent_vectors[i], sent_vectors[j])[0][0]

    return cosine_matrix

## Applying page rank algorithm
        - Page rank is generally used by google to rank web pages.
        - This algorithm can even be extended for text rank
        - That is giving ranks to each sentences and based on the rank, we can get the top N sentences that summarizes the document/documents.

In [12]:
def text_rank(cosine_matrix, sent_tokens, sent_as_key, top_n, top_n_percent):

    # get the similarity matrix graph
    nx_graph = nx.from_numpy_array(cosine_matrix)

    # gets the scores
    scores = nx.pagerank(nx_graph)

    # sort in descending order which has list of tuples [(score_1, sent_1), ...]
    sorted_score_sent = sorted((scores[i], sent) for i, sent in enumerate(sent_tokens))

    # sorted extracted top N sentences are random
    # we will again sort extracted top N sentences such that it follows the document sentence sequence
    sent_order_per_doc = {}
    for i in range(top_n):
        sent_order_per_doc[sent_as_key[sorted_score_sent[i][1]]] = sorted_score_sent[i][1]

    print('Top {} ({}%) extracted sentences are:\n'.format(top_n, top_n_percent))
    for k in sorted(sent_order_per_doc.keys()):
        print(sent_order_per_doc[k])

## Function to call all functions

In [16]:
def call_all_func(dfr, text_col, top_n_percent=30):

    # calling data_preparation
    sent_tokens, sent_as_key = data_preparation(dfr, text_col)

    # calling embed
    sent_vectors = embed(sent_tokens)

    # calling cosine_similarity_matrix
    cosine_matrix = cosine_similarity_matrix(sent_tokens, sent_vectors)

    top_n = int(len(sent_tokens)*(top_n_percent/100))

    # calling text_rank
    text_rank(cosine_matrix, sent_tokens, sent_as_key, top_n, top_n_percent)

In [17]:
text = ["""Swami Vivekananda a name that does not require any sort of introduction. He is an influential personality who is credited with enlightening the western world about Hinduism. He represented Hinduism in the Parliament of Religions in 1893 in Chicago and due to this an unknown monk of India suddenly leaped into fame. National Youth Day is observed on 12 January to commemorate the birth anniversary of Swami Vivekananda.

Swami Vivekananda founded Ramakrishna Mission on 1 May 1897 for one's own salvation and for the welfare of the world. Do you know his lectures, writings, letters, and poems are published as The Complete Works of Swami Vivekananda? He always focuses on teaching universal principles rather than personalities. He had tremendous intellect. His unique contributions always enlighten and awaken us. He was a spiritual leader and social reformer. 

"All the powers in the universe are already our. It is we who have put our hands before our eyes and cry that it is dark."- Swami Vivekananda

If anyone wants to study the origin of the Vedanta movement in America then study Swami Vivekananda travels across the US. He was a great thinker, great orator, and passionate patriot. It is not wrong to say that he was more than just a spiritual mind.

Born: 12 January, 1863

Place of Birth: Kolkata, India

Childhood Name: Narendranath Dutta

Father: Vishwanath Dutta

Mother: Bhuvaneshwari Devi

Education: Calcutta Metropolitan School; Presidency College, Calcutta

Religion: Hinduism

Guru: Ramakrishna

Founder of: Ramakrishna Mission (1897), Ramakrishna Math, Vedanta Society of New York

Philosophy: Advaita Vedanta

Literary works: Raja Yoga (1896), Karma Yoga (1896), Bhakti Yoga (1896), Jnana Yoga, My Master (1901), Lectures from Colombo to Almora (1897)

Death: 4 July, 1902

Place of Death: Belur Math, Belur, Bengal

Memorial: Belur Math. Belur, West Bengal

Swami Vivekananda was born on 12 January, 1863, Kolkata (earlier Calcutta). He was a spiritual leader and social reformer. His lectures, writings, letters, poems, ideas motivated not only the youth of India but also the whole world. He is the founder of Ramakrishna Mission and Belur Math in Calcutta, which are still working towards helping the needy. He was a man of wisdom and a very simple human being. 

"Arise, awake and stop not until the goal is achieved" - Swami Vivekananda

Swami Vivekananda: Life History and Education

Jagranjosh

Source: www. medium.com

Vivekananda childhood name was Narendranath Dutta, belonged to an affluent Bengali family in Calcutta. He was one of the eight children of Vishwanath Dutta and Bhuvneshwari Devi. On the occasion of Makar Sankranti, he was born on 12 January, 1863. His father was an attorney and an influential personality in society. Vivekananda's mother was a woman who has faith in God and has a great impact on his son.

At the age of eight in 1871, Vivekananda was enrolled at Ishwar Chandra Vidyasagar's Institution and later at the Presidency College in Calcutta. He was exposed to Western philosophy, Christianity, and science. He had an interest in music both instrumental as well as vocal. He was active in sports, gymnastics, wrestling, and bodybuilding. He was also fond of reading and till the time he had completed his graduation from the college he had acquired a vast knowledge of various subjects. Do you know on the one hand he read Hindu scriptures like Bhagavad Gita and the Upanishads and on the other hand western philosophies and spirituality by David Hume, Herbert Spencer etc?

“Be an atheist if you want, but do not believe in anything unquestioningly.”- Swami Vivekananda

Why is Swami Vivekananda's birthday celebrated as National Youth Day?

Spiritual Crisis and met with Ramakrishna Paramhansa

Jagranjosh

Source: www. swamishivapadananda.typepad.com

He had grown up in a religious family but studied several religious books and knowledge led him to question the existence of God and sometimes he believed in Agnosticism. But he could not completely deny the fact about the supremacy of God. In 1880, he joined Keshab Chandra Sen's Nava Vidhan and also became a member of Sadharan Brahmo Samaj led by Keshab Chandra Sen and Debendranath Tagore.

Brahmo Samaj recognized one God, unlike idol-worship. Several questions were running in the mind of Vivekananda and during his spiritual crisis, he first heard about Shri Ramakrishna from William Hastie, the Principal of the Scottish Church College. He finally met Shri Ramakrishna Paramhansa at Dakshineshwar Kali Temple and Vivekananda asked him a question, "Have you seen God?" which he had asked so many spiritual leaders but was not satisfied. But when he asked Ramakrishna, he gave such a simple answer that "Yes, I have. I see God as clearly as I see you, only in a much deeper sense". After this Vivekananda started visiting Dakshineshwar and got several answers to the questions that were in his mind.

When Vivekananda's father died, the whole family faced a financial crisis. He went to Ramakrishna and asked him to pray for his family but Ramakrishna refused and told Vivekananda to pray himself in front of Goddess Kali. He could not ask for wealth, money but instead of it, he asked for conscience and reclusion. That day he was marked with spiritual awakening and a way of ascetic life was started. This was the turning point in his life and accepted Ramakrishna as his Guru.

“Take risks in your life. If you win, you can lead, if you lose, you can guide.” Swami Vivekananda

In 1885, Ramakrishna developed throat cancer and was transferred to Calcutta and then later to a garden house in Cossipore. Vivekananda and other disciples of Ramakrishna took care of him. On 16 August, 1886, Shri Ramakrishna gave up his mortal body. Narendra was taught that the service to men was the most effective worship of God. After the demise of Ramakrishna, fifteen of his disciples including Narendranath began to live together at Baranagar in North Calcutta, which was named Ramakrishna Math. In 1887, all the disciples took vows of monkhood and Narendranath emerged as Vivekananda that is "the bliss of discerning wisdom." All of them performed yoga and meditation. Further, Vivekananda left the math and decided to tour the whole of India on foot which came to be known as 'Parivrajak'. He saw several social, cultural, and religious aspects of the people and also saw what common people faced in their daily life, their sufferings, etc.

Swami Vivekananda attended the World Parliament of Religions

Jagranjosh
Source: www. indiatvnews.com

When he came to know about the World Parliament that was organised in Chicago, America. He was keen to attend the meeting, to represent India and his Guru's philosophies. After various troubles, he attended the Religious meeting. On 11 September, 1893, he came upon the stage and stunned everyone while saying "My brothers and sisters of America". For this, he received a standing ovation from the audience. He described the principles of Vedanta, their spiritual significance, etc.
He stayed around two and a half years in America itself and founded the Vedanta Society of New York. He also travelled to the United Kingdom to preach the philosophies, spiritualism, and principles of Vedanta.

“Learn everything that is good from others but bring it in, and in your own way absorb it; do not become others.” Swami Vivekananda

Ramakrishna Mission and Vivekananda: Contribution to Social Reform

He founded Ramakrishna Mission

Around 1897, he returned to India and reached Calcutta where he founded Ramakrishna Mission on 1 May, 1897 at Belur Math. The goals of the mission were based on Karma Yoga and its main objective was to serve the poor and suffering or disturbed population of the country. Several social services are also performed under this mission like establishing schools, colleges, and hospitals. Teachings of Vedanta were also provided through conferences, seminars, and workshops, rehabilitation work across the country.

Let us tell you that Vivekananda's teachings were mostly based on Ramakrishna's spiritual teachings of Divine manifestations and his personal internalisation of the Advaita Vedanta Philosophy. According to him, the ultimate goal of life is to achieve the freedom of the soul and that encompasses the entirety of one's religion.

Death

He predicted that he will not live till the age of 40. Therefore, on 4 July, 1902, he died while doing meditation. He is said to have attained 'Mahasamadhi' and was cremated on the Banks of the river Ganga.

“A man is not poor without a rupee but a man is really poor without a dream and ambition.” Swami Vivekananda

Key Works of Swami Vivekananda

- The Complete Works of Swami Vivekananda

- Swami Vivekananda’s Speeches at the Parliament of Religions, Chicago, 1893

- Letters of Swami Vivekananda

- Jnana Yoga: The Yoga of Knowledge

- Yoga: The Yoga of Love and Devotion

- Yoga: The Yoga of Action

- Raja Yoga: The Yoga of Meditation

Key Works on Swami Vivekananda

- Vivekananda A Biography, by Swami Nikhilananda

- Swami Vivekananda by Eastern and Western Disciples

- The Master As I Saw Him, by Sister Nivedita

- Reminiscences of Swami Vivekananda

- The Life of Vivekananda, by Romain Rolland

No doubt Swami Vivekananda's teachings not only motivated the youth but also the whole world. He laid the true foundations of India's unity as a nation. He taught us how to live together with so many diversities. He was successful in constructing a virtual bridge between the culture of the East and the West. He played a key role in isolating India's culture from the rest of the World.

“Take up one idea, make that one idea your life, think of it, dream of it, let the brain, muscles, nerves, every part of your body be full of that idea, and just leave every other idea alone. This is the way to success.” Swami Vivekananda"""]

df = pd.DataFrame({'article_text':text})
df

,article_text
0,Swami Vivekananda a name that does not require...


In [18]:
call_all_func(df, 'article_text')

Top 25 (30%) extracted sentences are:

He always focuses on teaching universal principles rather than personalities.
"All the powers in the universe are already our.
It is we who have put our hands before our eyes and cry that it is dark.
medium.com

Vivekananda childhood name was Narendranath Dutta, belonged to an affluent Bengali family in Calcutta.
On the occasion of Makar Sankranti, he was born on 12 January, 1863.
Vivekananda's mother was a woman who has faith in God and has a great impact on his son.
He had an interest in music both instrumental as well as vocal.
He was active in sports, gymnastics, wrestling, and bodybuilding.
But he could not completely deny the fact about the supremacy of God.
Brahmo Samaj recognized one God, unlike idol-worship.
But when he asked Ramakrishna, he gave such a simple answer that "Yes, I have.
I see God as clearly as I see you, only in a much deeper sense".
When Vivekananda's father died, the whole family faced a financial crisis.
He could not as